# Assignment 2
Emma McCready\
sba23001

---

This dashboard was designed in a separate notebook to the Machine Learning Analysis (found in Codebook 2), as I don't currently have access to the PC I did the original ML analysis on, and I don't want to run or edit the codebook as I'm using a mac, and so the results will be different from when I analysed the results. Therefore, this output of this dashboard will differ from the results in Codebook 2.

The code from the best performing ML model (Random Forest Regression) was copied and pasted into this Codebook.

---

References:

[Article: Build a Machine Learning simulation tool with Dash](https://towardsdatascience.com/build-a-machine-learning-simulation-tool-with-dash-b3f6fd512ad6?gi=b773c179715f)  
[Article: Deploy Machine Learning Model Using Python Dash and Pipenv](https://towardsdatascience.com/deploy-machine-learning-model-using-dash-and-pipenv-c543569c33a6)  
[Article: The quickest way to build Dashboards for Machine Learning models](https://towardsdatascience.com/the-quickest-way-to-build-dashboards-for-machine-learning-models-ec769825070d)


In [1]:
#!pip install dash
#!pip install dash_daq
#!pip upgrade explainerdashboard

In [2]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore') 

### ML Libraries:
from sklearn import preprocessing
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict, 
                                     StratifiedKFold, learning_curve, train_test_split)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import (r2_score, mean_squared_error, mean_absolute_error,explained_variance_score, 
                             classification_report, confusion_matrix)
from statsmodels.tools.eval_measures import mse, rmse

### For Dashboard:
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq
from dash.dependencies import Input, Output

#### First I will import my dataset. 
The exact same code has been copied from Codebook 2.

In [3]:
files = glob.glob('EUROSTAT*.csv')
EUROSTAT_data = pd.DataFrame()  # to store merged data
loaded_files = []  # so the list of files loaded in can be seen in the correct order

for i, f in enumerate(files):
    dataset = pd.read_csv(f) 
    
    obs_columns = [col for col in dataset.columns if col.startswith('OBS_VALUE')]
    dataflow_value = dataset['DATAFLOW'].iloc[0]  # i used 0 just because it's constant in the dataset anyway
    rename_mapping = {obs_column: f'{dataflow_value}_{obs_column.split("_")[-1]}' for obs_column in obs_columns} # so that the output will be "[survey code]_VALUE"
    dataset.rename(columns=rename_mapping, inplace=True)
    
    unit_columns = [col for col in dataset.columns if col.startswith('unit')]
    rename_mapping_unit = {unit_column: f'{dataflow_value}_{unit_column.split("_")[-1]}' for unit_column in unit_columns} 
    dataset.rename(columns=rename_mapping_unit, inplace=True)
    
    filter_cols = ['accident', 'isced11', 'c_resid'] 
    for col in filter_cols:
        if col in dataset.columns:
            dataset = dataset[dataset[col] == 'TOTAL'] # to keep just the total values

    loaded_files.append(f)
    print(f'Loaded file: {f}')

    if EUROSTAT_data.empty:
        EUROSTAT_data = dataset
    else:
        suffixes = ('', f'_{i}')  # to solve merge error issue
        EUROSTAT_data = pd.merge(EUROSTAT_data, dataset, on=['geo', 'TIME_PERIOD'], how='outer', suffixes=suffixes)

Loaded file: EUROSTAT_Average rating of satisfaction by domain, sex, age and educational attainment level_TIMSAT.csv
Loaded file: EUROSTAT_Average rating of satisfaction by domain, sex, age and educational attainment level_LIVENVSAT.csv
Loaded file: EUROSTAT_Average rating of satisfaction by domain, sex, age and educational attainment level_RELSAT.csv
Loaded file: EUROSTAT_railwaylength.csv
Loaded file: EUROSTAT_Nights spent at tourist accommodation establishments.csv
Loaded file: EUROSTAT_Average rating of satisfaction by domain, sex, age and educational attainment level_JOBSAT.csv
Loaded file: EUROSTAT_Average rating of satisfaction by domain, sex, age and educational attainment level_COMSAT.csv
Loaded file: EUROSTAT_Average rating of satisfaction by domain, sex, age and educational attainment level_ACCSAT.csv
Loaded file: EUROSTAT_passengers transported_Total.csv
Loaded file: EUROSTAT_Passenger train traffic performance by type of train speed.csv
Loaded file: EUROSTAT_At-risk-of-pov

In [9]:
columns_to_drop = EUROSTAT_data.columns[EUROSTAT_data.columns.str.startswith('DATAFLOW') | 
                                        EUROSTAT_data.columns.str.startswith('LAST') | 
                                        EUROSTAT_data.columns.str.startswith('freq') |
                                        EUROSTAT_data.columns.str.startswith('OBS_FLAG') | 
                                        EUROSTAT_data.columns.str.startswith('sex') |
                                        EUROSTAT_data.columns.str.startswith('age') |
                                        EUROSTAT_data.columns.str.startswith('indic_il') |
                                        EUROSTAT_data.columns.str.startswith('nace_r2') |
                                        EUROSTAT_data.columns.str.startswith('vehicle') |
                                        EUROSTAT_data.columns.str.startswith('tra_infr') |
                                        EUROSTAT_data.columns.str.startswith('n_tracks') |
                                        EUROSTAT_data.columns.str.startswith('indic_wb') |
                                        EUROSTAT_data.columns.str.startswith('isced11') |
                                        EUROSTAT_data.columns.str.startswith('accident') |
                                        EUROSTAT_data.columns.str.startswith('VALUE') |
                                        EUROSTAT_data.columns.str.startswith('c_resid') |
                                        EUROSTAT_data.columns.str.startswith('hotelsize')
                                       ]

EUROSTAT_clean = EUROSTAT_data.drop(columns=columns_to_drop)

import re
EUROSTAT_clean.rename(columns=lambda x: re.sub(r'ESTAT:|[(]\d+\.\d+[)]', '', x), inplace=True)
EUROSTAT_clean.rename(columns=lambda x: re.sub(r'.VALUE', '', x), inplace=True) 

filtered_columns = EUROSTAT_clean.filter(like='unit')
EUROSTAT_clean = EUROSTAT_clean.drop(columns=filtered_columns.columns)
EUROSTAT_supervised = EUROSTAT_clean[EUROSTAT_clean['TIME_PERIOD'] >= 2004]
EUROSTAT_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   geo               953 non-null    object 
 1   TIME_PERIOD       953 non-null    int64  
 2   ILC_PW01          54 non-null     float64
 3   ILC_PW01_1        27 non-null     float64
 4   ILC_PW01_2        54 non-null     float64
 5   TTR00003          300 non-null    float64
 6   TOUR_OCC_NINATS   295 non-null    float64
 7   ILC_PW01_5        54 non-null     float64
 8   ILC_PW01_6        27 non-null     float64
 9   ILC_PW01_7        27 non-null     float64
 10  RAIL_PA_TOTAL     431 non-null    float64
 11  RAIL_TF_PASSMOV   783 non-null    float64
 12  ILC_LI02          615 non-null    float64
 13  ILC_PW01_11       27 non-null     float64
 14  ILC_PW01_12       108 non-null    float64
 15  ILC_PW01_13       27 non-null     float64
 16  RAIL_PA_TOTAL_14  430 non-null    float64
 1

Now that the dataset is ready, I will preprocess it in the same way as Codebook 2:

In [5]:
EUROSTAT_supervised_dummy = pd.get_dummies(EUROSTAT_supervised, drop_first=True) # only 1 cat feature so wont bother defining. whether or not to use `drop_first=True` TBD

X = EUROSTAT_supervised_dummy.drop(['RAIL_PA_TOTAL_14'], axis=1)
y = EUROSTAT_supervised_dummy['RAIL_PA_TOTAL_14']

y = y.dropna() # had issues w missing values in this set
X = X.loc[y.index] # matching index to align the two datasets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # random state for reproducibility

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # only need to appy this to input features to help ML algorithm be more efficient
X_test_scaled = scaler.transform(X_test)


knn_imputer = KNNImputer()
X_train_imputed = pd.DataFrame(knn_imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(knn_imputer.transform(X_test), columns=X_test.columns)

Now I'll fit the model:

In [6]:
regressor = RandomForestRegressor(random_state = 0)

param_grid = {'max_depth': [1, 10, 100, 150], 'n_estimators': [100, 125, 150]} 
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train_imputed, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

best_model.fit(X_train_imputed, y_train)

y_pred_random = best_model.predict(X_test_imputed)

print(best_params)
print(best_model)

print("R-squared of the model in training set is: {}".format(best_model.score(X_train_imputed, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(best_model.score(X_test_imputed, y_test)))
print("Root mean squared error of the prediction is: {}".format(mse(y_test, y_pred_random)**(1/2)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_pred_random) / y_test)) * 100))

{'max_depth': 10, 'n_estimators': 150}
RandomForestRegressor(max_depth=10, n_estimators=150, random_state=0)
R-squared of the model in training set is: 0.9989447715068346
-----Test set statistics-----
R-squared of the model in test set is: 0.9928063649785939
Root mean squared error of the prediction is: 1850.7931389666421
Mean absolute percentage error of the prediction is: 8.500478047669901


Interestingly, when run on Mac OS X, the model actually performs better than when run on Windows.

**Previous results (from Codebook 2):**  
{'max_depth': 100, 'n_estimators': 100}  
RandomForestRegressor(max_depth=100, random_state=0)  
R-squared of the model in training set is: 0.999042565612404  
-----Test set statistics-----    
R-squared of the model in test set is: 0.9899003360806337  
Root mean squared error of the prediction is: 54063.985747570376  
Mean absolute percentage error of the prediction is: 10.826072578458913  

**New results:**
{'max_depth': 10, 'n_estimators': 150}  
RandomForestRegressor(max_depth=10, n_estimators=150, random_state=0)  
R-squared of the model in training set is: 0.9989447715068346  
-----Test set statistics-----  
R-squared of the model in test set is: 0.9928063649785939  
Root mean squared error of the prediction is: 1850.7931389666421  
Mean absolute percentage error of the prediction is: 8.500478047669901  

**In the model run on this Codebook, it has a higher R squared value and a lower RMSE and Mean absolute percentage error, implying it's a better fit and more accurate.**

---

for the dashboard, I'd like to focus on the feature importance:

In [7]:
feature_importance = best_model.feature_importances_
features = X_train_imputed.columns

I tried the ExplainerDashboard library as suggested in the referenced articles but I got an error saying I had the wrong version, so I changed the version of the library and it still wouldn't work. So I abandoned this approach.

In [19]:
feature_importance = best_model.feature_importances_ # so I can assess feature importance and plot it
features = X_train_imputed.columns

feature_display_names = { #to make plot labels easier to interpret
    'ILC_PW01': 'TIMSAT',
    'ILC_PW01_1': 'LIVENVSAT',
    'ILC_PW01_2': 'RELSAT',
    'ILC_PW01_5': 'JOBSAT',
    'ILC_PW01_6': 'COMSAT',
    'ILC_PW01_7': 'ACCSAT',
    'ILC_PW01_11': 'GREENSAT',
    'ILC_PW01_12': 'LIFESAT',
    'ILC_PW01_13': 'MEANLIFE',
    'ILC_PW01_15': 'FINSAT'
}

features_display = [feature_display_names.get(feature, feature) for feature in features] #mapping names to features

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Tabs([
        # Tab 1 over view tab
        dcc.Tab(label='Overview', children=[
            html.Div([
                dcc.Markdown('''
                    # Machine learning model for prediction of the amount of passengers travelling on a train in a given year for a given member state
                    
                    ##### The features in this dataset were named based on their survey codes in the EUROSTAT database.
                    * **ACCSAT** | Satisfaction with accommodation
                    * **COMSAT** | Satisfaction with commuting time
                    * **FINSAT** | Satisfaction with financial situation
                    * **GREENSAT** | Satisfaction with recreational and green areas
                    * **JOBSAT** | Job satisfaction
                    * **LIFESAT** | Overall life satisfaction
                    * **LIVENVSAT** | Satisfaction with living environment
                    * **MEANLIFE** | Meaning of life
                    * **RELSAT** | Satisfaction with personal relationships
                    * **TIMSAT** | Satisfaction with time use
                    * **TOUR_OCC_NINATS** | Nights spent at hotels and similar accommodation (NACE Rev.2 activity I55.1) by size class
                    * **RAIL_TF_PASSMOV** | Passenger train traffic performance by type of train speed
                    * **RAIL_PA_TOTAL** | Passengers transported measured in millions of passenger-kilometres
                    * **RAIL_AC_CATNMBR** | Railway accidents by type of accident - annual data (2004-2015)
                    * **TTR00003** | Total length of railway lines
                    
                '''),
            ])
        ]),
        
        # Tab 2
        dcc.Tab(label='Feature Importance', children=[
            html.Div([
                html.H1("Random Forest Feature Importance Dashboard"),
                dcc.Dropdown(
                    id='feature-dropdown',
                    options=[{'label': feat, 'value': feat} for feat in features_display],
                    value=features_display[0],  # Initial feature to display
                    multi=False
                ),
                dcc.Graph(id='feature-importance-plot'),
            ])
        ]),
        
        # Tab 3
        dcc.Tab(label='Actual vs. Predicted Plots', children=[
            html.Div([
                html.H1("Actual vs. Predicted Plots"),
                dcc.Graph(id='actual-predicted-plot'),
            ])
        ]),
    ])
])


@app.callback(
    Output('feature-importance-plot', 'figure'),
    [Input('feature-dropdown', 'value')]
)
def update_feature_importance_plot(selected_feature):
    # Create a DataFrame with feature importance data
    feature_importance_df = pd.DataFrame({
        'Feature': features_display,
        'Importance': feature_importance
    })
    feature_importance_df = feature_importance_df[~feature_importance_df['Feature'].str.startswith('geo')]
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    fig = px.bar(feature_importance_df, x='Importance', y='Feature', orientation='h',
                 title=f'Feature Importance - {selected_feature}')
    fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
    fig.update_xaxes(showline=True, linecolor='black', linewidth=1, mirror=True)
    fig.update_yaxes(showline=True, linecolor='black', linewidth=1, mirror=True)
    

    return fig


@app.callback(
    Output('actual-predicted-plot', 'figure'),
    [Input('feature-dropdown', 'value')]
)
def update_actual_predicted_plot(selected_feature):
    fig = px.scatter(x=y_test, y=best_model.predict(X_test_imputed), labels={'x': 'Actual Passenger Count (millions)', 'y': 'Predicted Passenger Count (millions)'},
                     title='Actual vs. Predicted Plot')
    fig.update_layout(plot_bgcolor='white', paper_bgcolor='white') # minimising data ink
    fig.update_xaxes(showline=True, linecolor='black', linewidth=1, mirror=True) #plots weren't easy to read otherwise
    fig.update_yaxes(showline=True, linecolor='black', linewidth=1, mirror=True)
    fig.update_layout(showlegend=False)

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

# Conclusions

As expected, the amount of passengers travelling per km has the highest importance when predicting passenger counts. 
from actual v predicted plot, it wasn't too far off which is good
